In [1]:
import pandas as pd
import numpy as np
import json

In [4]:
# Прочитать файл с героями
heroes = pd.read_csv('All_heroes.csv').drop(['id.1'], axis=1)
# Прочитать файл с нормальными id команд
teams = json.load(open('ID_nameTeam (name main) (correct)'))
# Матчи с переиндексацией
matches = pd.read_csv('table wtih team, kills, picks path 7.06.csv', index_col=0, 
                      encoding='latin-1').reset_index().drop(['index'], axis=1)


In [5]:
test_matches = matches[:5]
#matches['radiant_name'].value_counts() + matches['dire_name'].value_counts()
test_matches

,match_id,start_time,radiant_team_id,radiant_score,dire_team_id,dire_score,radiant_win,radiant_H1,radiant_H2,radiant_H3,...,radiant_P2,radiant_P3,radiant_P4,radiant_P5,dire_P1,dire_P2,dire_P3,dire_P4,dire_P5,league_name
0,2897064589,1483677730,1883502.0,23,350190.0,12,True,86.0,8.0,76.0,...,132851371.0,106573901.0,92423451.0,106809101.0,89871557.0,181716137.0,112377459.0,121052479.0,93616251.0,ESL One Genting 2017
1,2897074199,1483678249,3214108.0,8,1375614.0,37,False,21.0,55.0,71.0,...,86726887.0,87196890.0,40547474.0,47434686.0,139876032.0,117281554.0,82327674.0,149486894.0,87012746.0,ESL One Genting 2017
2,2897155244,1483682366,2581813.0,16,2512249.0,26,False,79.0,63.0,28.0,...,180012313.0,116782914.0,152545459.0,102099826.0,87382579.0,103735745.0,38628747.0,86700461.0,86725175.0,ESL One Genting 2017
3,2897190211,1483683976,1836806.0,24,2659468.0,26,True,20.0,11.0,28.0,...,101586543.0,118134220.0,102644565.0,111114687.0,119576842.0,126417273.0,94296097.0,101525357.0,113457795.0,ESL One Genting 2017
4,2897294037,1483688368,1883502.0,16,1375614.0,35,False,87.0,48.0,8.0,...,132851371.0,106573901.0,92423451.0,106809101.0,139876032.0,149486894.0,117281554.0,82327674.0,87012746.0,ESL One Genting 2017


In [6]:
heroes[:2]

,id,attack_type,localized_name,primary_attr,roles,Disabler,Nuker,Carry,Initiator,Escape,Durable,Support,Pusher,Jungler
0,1,Melee,Anti-Mage,agi,"['Carry', 'Escape', 'Nuker']",0,1,1,0,1,0,0,0,0
1,2,Melee,Axe,str,"['Initiator', 'Durable', 'Disabler', 'Jungler']",1,0,0,1,0,1,0,0,1


In [9]:
# Создвание списка имен героев и списка имен команд и объединение его
name_heroes = heroes['localized_name'].values

a = []
for i in teams.keys():
    a.append(i)
name_team = np.array(a)

name_columns = np.concatenate((name_team, name_heroes), axis=0)
#name_columns

In [83]:
# основной df c которым потом работать, в нем так же будут записаны килы и радинт_вин  
df_features_name = pd.DataFrame(columns=name_heroes)
# df из которого сотавляется новый df
little = matches
# Собрать все команды и героев
for index in range(len(little)):
    # Заполнение команд
    df_features_name.loc[index, little['radiant_name'].loc[index]] = 1
    df_features_name.loc[index, little['dire_name'].loc[index]] = -1
    # radiant
    for i in range(1,6):
        id_her = little['radiant' + str(i)].loc[index]
        name_her = heroes['localized_name'][heroes['id'] == id_her].item()
        df_features_name.loc[index, name_her] = 1
    # dire
    for i in range(1,6):
        id_her = little['dire' + str(i)].loc[index]
        name_her = heroes['localized_name'][heroes['id'] == id_her].item()
        df_features_name.loc[index, name_her] = -1     
df_features_name['kills'] = little['kills']
df_features_name['radiant_win'] = little['radiant_win']
df_features_name = df_features_name.fillna(0)

In [96]:
columns_team = df_features_name.loc[:, 'CAVALRY':].columns

In [97]:
df_features_name.to_csv('table with features teams, heroes & kills (all team) 7.06.csv')